In [ ]:
import adaptive
import numpy as np
import scipy
adaptive.notebook_extension()

from adaptive.learner.learnerND import triangle_loss, volume, uses_nth_neighbors, curvature_loss_function
import holoviews as hv
hv.notebook_extension()
hv.extension('bokeh')

import scipy.misc

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def curvature_loss_function(beta=0.05):
    # XXX: add doc-string!
    @uses_nth_neighbors(1)
    def curvature_loss(simplex, values, neighbors, neighbor_values):
        """Compute the curvature loss of a simplex.

        Parameters
        ----------
        simplex : list of tuples
            Each entry is one point of the simplex.
        values : list of values
            The function values of each of the simplex points.
        neighbors : list of tuples
            The neighboring points of the simplex, ordered such that simplex[0]
            exacly opposes neighbors[0], etc.
        neighbor_values : list of values
            The function values for each of the neighboring points.

        Returns
        -------
        loss : float
        """
        dim = len(simplex[0]) # the number of coordinates
        V_s = volume(simplex)

        loss_curvature = triangle_loss(simplex, values, neighbors, neighbor_values)
        return ((loss_curvature + beta * V_s ** ((2+dim)/dim)))
    return curvature_loss


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    r = (x**2+y**2)**0.5
    return np.exp(- (r-0.3)**2 / 0.03**2)


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    r = (x**2+y**2)**0.5
    return np.sinc(r*10)


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    r = ((x-0.2)**2+(y-0.4)**2)**0.5
    return np.exp(-r**2 / 0.05**2)


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    r = (x**2+y**2)**0.5
    return r


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    x = x*2
    y = y*2
    return -np.sin(np.pi*x) * np.exp(- y**2/0.3**2)


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    return x**2+y**2


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    x = x*2
    y = y*2
    return np.tanh(5*(x+y))


In [ ]:
bounds = [(-.5, .5)] * 2
def f(xy):
    x,y = xy
    a = 0.2
    return x + np.exp(-(4*(x**2 + y**2) - 0.75**2)**2/a**4)


In [ ]:
def simple_runner(learner, goal):
    learner.xx = []
    learner.ls = []
    while not goal(learner):
        x = learner.ask(1)[0][0]
        y = learner.function(x)
        learner.tell(x,y)
        learner.xx.append(x)
        learner.ls.append(learner.loss())

In [ ]:
def integrate_err(e, norm=2, bounds=None):
    bd = np.transpose(bounds)
    x = np.random.random((1000000,len(bd))) * (-np.subtract(*bd)) + bd[0]
    I = np.average(np.abs(e(x)) ** norm) 
    A = np.abs(np.product(np.subtract(*bd)))
    return (I * A) ** (1/norm)

def get_xy(learner, N):
    xx = learner.xx[:N]
    yy = [learner.data[x] for x in xx]
    return xx, yy

def get_interpolation_function(learner, n):
    x, y = get_xy(learner, n)
    lin = scipy.interpolate.LinearNDInterpolator(x,y)
    return lin

def get_error_function(learner, n):
    ip = get_interpolation_function(learner, n)
    return (lambda x: ip(x) - f(x.T))

def get_err(learner, n, norm=2, bounds=None):
    e = get_error_function(learner, n)
    return integrate_err(e, norm, bounds or learner._bbox)

def get_errs(ns, learner, norm=2):
    return [get_err(learner, n, norm=norm) for n in ns]



In [ ]:
loss = curvature_loss_function(0.05)

In [ ]:
N = 8000
learner = adaptive.LearnerND(f, bounds=bounds, loss_per_simplex=loss)
simple_runner(learner, goal=lambda l: l.npoints==N)


In [ ]:
learner.plot(tri_alpha=0.4)

In [ ]:
learner_no_curv = adaptive.LearnerND(f, bounds=bounds, loss_per_simplex=None)
simple_runner(learner_no_curv, goal=lambda l: l.npoints==N)


In [ ]:
NP = 50

In [ ]:
learner._bbox

In [ ]:
Na = np.unique(np.logspace(0.5, np.log10(N), NP, dtype=int))
errs_A = []
for n in Na:
    loss = learner.ls[n-1]
    err = get_err(learner=learner, n=n, norm=1)
    errs_A.append(err)
    print(n, end=" ")

In [ ]:
errs_Anc = []
for n in Na:
    loss = learner_no_curv.ls[n-1]
    err = get_err(learner=learner_no_curv, n=n, norm=1)
    errs_Anc.append(err)
    print(n, end=" ")

In [ ]:
Nh = np.unique(np.logspace(0.5, np.log10(N**0.5), NP, dtype=int))
NH = Nh**2
errs_H = []
for n in Nh:
    ax = [np.linspace(*b, n) for b in bounds]
    xx = np.meshgrid(*ax)
    x = np.array(list(zip(*[i.flat for i in xx])))
    y = f(x.T)

    lin = scipy.interpolate.LinearNDInterpolator(x,y)
    e = (lambda x: lin(x) - f(x.T))
    I = integrate_err(e, norm=1, bounds=bounds)
    errs_H.append(I)
    print(n**2, end=" ")

In [ ]:
# Fit for adaptive
NNN_mask = np.array(Na) > 4000
NNN_ok = np.array(Na)[NNN_mask]
err_A_ok = np.array(errs_A)[NNN_mask]
xx = np.log(NNN_ok-1)
yA = np.log(err_A_ok)

nn2 = np.logspace(np.log10(100), np.log10(N))
p, c = np.polyfit(xx, yA, 1)
err_A_f = np.exp(c) * nn2 ** p
print(f"Adaptive: fit: {np.exp(c):.2f} * n ^ {p:.3f}")

c1 = np.polyfit(xx, err_A_ok * NNN_ok, 0)[0]
err_A_f2 = c1 * nn2 ** -1
print(f"Adaptive, fixed fit: {c1:.2f} * N ^ -1\n")


# Fit for adaptive no curv
NNN_mask = np.array(Na) > 4000
NNN_ok = np.array(Na)[NNN_mask]
err_Anc_ok = np.array(errs_Anc)[NNN_mask]
xx = np.log(NNN_ok-1)
yAnc = np.log(err_Anc_ok)

nn2 = np.logspace(np.log10(100), np.log10(N))
p, c = np.polyfit(xx, yAnc, 1)
err_Anc_f = np.exp(c) * nn2 ** p
print(f"Adaptive no curv: fit: {np.exp(c):.2f} * n ^ {p:.3f}")

c1 = np.polyfit(xx, err_Anc_ok * NNN_ok, 0)[0]
err_Anc_f2 = c1 * nn2 ** -1
print(f"Adaptive no curv, fixed fit: {c1:.2f} * N ^ -1\n")




# Fit for homogenous
NNN_mask = np.array(NH) > 4000
NNN_ok = np.array(NH)[NNN_mask]
err_H_ok = np.array(errs_H)[NNN_mask]
xx = np.log(NNN_ok-1)
yH = np.log(err_H_ok)

p, c = np.polyfit(xx, yH, 1)
err_H_f = np.exp(c) * nn2 ** p
print(f"Homogenous: fit: {np.exp(c):.2f} * n ^ {p:.3f}")

c1 = np.polyfit(xx, err_H_ok * NNN_ok, 0)[0]
err_H_f2 = c1 * nn2 ** -1
print(f"Homogenous, fixed fit: {c1:.2f} * N ^ -1")



In [ ]:
%%opts Scatter Curve [logx=True, logy=True, width=500, height=500]
%%opts Scatter (marker='o' size=4)

hv.Scatter((NNN_ok, err_H_ok * NNN_ok))

c1 = np.polyfit(xx, err_H_ok * NNN_ok, 0)

c2 = np.average(err_H_ok * NNN_ok)

print(c1, c2)

In [ ]:
%%opts Scatter Curve [logx=True, logy=True, width=500, height=500]
%%opts Scatter (marker='o' size=4)


(
    hv.Scatter((Na, errs_A)).relabel("Adaptive") *
    hv.Scatter((NH, errs_H)).relabel("Homogenous") *
    
    hv.Scatter((NH, errs_H)).relabel("Adaptive no curv") *
    
    hv.Curve((nn2, err_A_f)).relabel("Adaptive fit")  * 
    hv.Curve((nn2, err_H_f)).relabel("Homogenous fit") *
    
    hv.Curve((nn2, err_A_f2)).relabel("Adaptive forced")  * 
    hv.Curve((nn2, err_H_f2)).relabel("Homogenous forced")
).redim(x="N points", y="Error")


In [ ]:
%%opts Scatter Curve [logx=True, logy=True, width=500, height=500]
%%opts Scatter (marker='o' size=4)


fig = plt.figure(figsize=(5,5))
ax = plt.gca()

# ax.loglog(nn2, err_A_f)
# ax.loglog(nn2, err_H_f)

# ax.scatter(Na, errs_A, s=5)
# ax.scatter(NH, errs_H, s=5)
# 

ax.loglog(NH, errs_H, marker='o', linewidth=0, markersize=3)
ax.loglog(Na, errs_A, marker='o', linewidth=0, markersize=3)
ax.loglog(Na, errs_Anc, marker='o', linewidth=0, markersize=3)


ax.legend(['Homogeneous', 'Adaptive, curvature loss',  'Adaptive, area loss'])
plt.xlabel('Number of points')
plt.ylabel('Error')
plt.savefig('error.svg')

In [ ]:
import numpy as np
import scipy.spatial

import plotly.offline as py
import plotly.figure_factory as FF
import plotly.graph_objs as go

py.init_notebook_mode()

In [ ]:
X, Y = learner.points.T
Z = [learner.data[tuple(p)] for p in learner.points]
tri = scipy.spatial.Delaunay(points=learner.points)

In [ ]:
fig1 = FF.create_trisurf(x=X, y=Y, z=Z,
                         simplices=tri.simplices)

py.iplot(fig1)

In [ ]:
plt.plot(Na, Na * errs_A)
plt.plot(NH, NH * errs_H)
plt.legend(['Adaptive', 'Homogeneous'])

In [ ]:
Nh**2